In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from pygame import mixer
import numpy as np

pygame 2.0.2 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')
model=load_model(r'C:\Users\ganes\Desktop\drowsiness\mrlEyes_2018_01\model\model.h5')

In [19]:
mixer.init()
sound=mixer.Sound(r'C:\Users\ganes\Desktop\drowsiness\mrlEyes_2018_01\alarm.wav')
cap=cv2.VideoCapture(0)
score=0
while True:
    ret,frame=cap.read()
    height,width=frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    eyes=eye_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=1)
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color=(255,0,0),thickness=3)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh),color=(255,0,0),thickness=3)
        #preprocesing\steps 
        eye=frame[ey:ey+eh,ex:ex+w]
        eye=cv2.resize(eye,(80,80))
        eye=eye/255
        eye=eye.reshape(80,80,3)
        eye=np.expand_dims(eye,axis=0)
        #preprocseeing is done model prediction
        pred=model.predict(eye)
        #print(pred)
        if pred[0][0]>0.30:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255)
                        ,thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'score'+str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255)
                        ,thickness=1,lineType=cv2.LINE_AA)
            score+=1
            if(score>3):
                try:
                    sound.play()
                except:
             
                    pass
        if pred[0][1]>0.50:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255)
                        ,thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'score'+str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255)
                        ,thickness=1,lineType=cv2.LINE_AA)
            score-=1
            if score<0:
                score=0
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break


cap.release() 
cv2.destroyAllWindows()
    